## Connexion postgres

In [9]:
import os
from dotenv import load_dotenv
import psycopg2

# Charger les variables d'environnement depuis le fichier .env
load_dotenv(".env")

USER_PSQL = os.environ.get("POSTGRESQL_LOCAL_USER")
PASSWORD_PSQL = os.environ.get("POSTGRESQL_LOCAL_PASSWORD")

# Se connecter à la base de données PostgreSQL
conn = psycopg2.connect(user=USER_PSQL, password=PASSWORD_PSQL, host="localhost", port="5432", database="app_auth")
cursor = conn.cursor()

## Création de la table

In [3]:

create_table = f'CREATE TABLE "user_for_comparison" (' \
   f' user_id INT PRIMARY KEY,' \
    f'firstname VARCHAR(255),' \
    f'lastname VARCHAR(255),' \
    f'email VARCHAR(255) UNIQUE,' \
    f'username VARCHAR(255) UNIQUE CHECK(LENGTH(username) > 8),' \
    f'password VARCHAR(255) CHECK(LENGTH(password) > 8),' \
   f' created_at TIMESTAMP);' 

cursor.execute(create_table)

## Création des 100 000 utilisateurs pour comparaison

In [4]:
import time
import random
from faker import Faker
from psycopg2.extras import execute_batch


# Fonction pour générer une date aléatoire dans une plage
def random_date(start, end, prop):
    stime = time.mktime(time.strptime(start, '%m/%d/%Y %I:%M %p'))
    etime = time.mktime(time.strptime(end, '%m/%d/%Y %I:%M %p'))
    ptime = stime + prop * (etime - stime)
    
    # Utiliser le format ISO 8601 pour la date/heure
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(ptime))

# Utiliser Faker pour générer des données aléatoires
fake = Faker()

# Dates de début et de fin pour la période
start_date = "01/01/2023 12:00 AM"
end_date = "06/30/2023 11:59 PM"

total_users = 100000
batch_size = 1000
cpt = 1


# Générer et insérer les utilisateurs avec des données crédibles
user_data = []
for _ in range(total_users):
    print(cpt)
    first_name = fake.first_name()
    last_name = fake.last_name()

      # Générer un nom d'utilisateur avec au moins 8 caractères
    while True:
        username = fake.unique.user_name()
        # Exécuter la requête SELECT pour vérifier si le nom d'utilisateur est déjà utilisé
    
        if len(username) > 8:
            break


 
    email = fake.unique.email()
        

    # Générer un nom de mot de passe avec au moins 8 caractères
    while True:
        password = fake.password()
        if len(password) > 8:
            break

    # Générer une date de création aléatoire dans la période spécifiée
    created_at = random_date(start_date, end_date, random.random())

    # Ajouter les données à la liste
    user_data.append({
        'user_id': cpt,
        'first_name': first_name,
        'last_name': last_name,
        'email': email,
        'username': username,
        'password': password,
        'created_at': created_at
    })

    # Incrémenter le compteur pour le user_id
    cpt += 1

    # Si la taille de la liste atteint le batch_size, insérer le lot dans la base de données
    if len(user_data) >= batch_size:
        execute_batch(cursor, """
            INSERT INTO "user_for_comparison" (user_id, firstname, lastname, email, username, password, created_at)
            VALUES (%(user_id)s, %(first_name)s, %(last_name)s, %(email)s, %(username)s, %(password)s, %(created_at)s);
        """, user_data)
        user_data = []

# Si des données restent dans la liste, insérer le dernier lot
if user_data:
    execute_batch(cursor, """
        INSERT INTO "user_for_comparison" (user_id, firstname, lastname, email, username, password, created_at)
        VALUES (%(user_id)s, %(first_name)s, %(last_name)s, %(email)s, %(username)s, %(password)s, %(created_at)s);
    """, user_data)


# Valider les changements
conn.commit()


In [5]:
# Récupérer 10 utilisateurs de la table
cursor.execute('SELECT username FROM "user_for_comparison" LIMIT 5')
usernames_to_query = [row[0] for row in cursor.fetchall()]

for username in usernames_to_query:
    # Créer un nouveau curseur à chaque itération
    cursor = conn.cursor()

    query = f"SELECT * FROM user_for_comparison WHERE username = '{username}'"

    print(f"Temps d'exécution pour {username}:")
    %timeit -r 1 -o cursor.execute(query) 

    results = cursor.fetchall()

    # Fermer le curseur après chaque itération
    cursor.close()


Temps d'exécution pour leejessica:
379 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1,000 loops each)
Temps d'exécution pour peterdougherty:
406 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1,000 loops each)
Temps d'exécution pour victoria59:
363 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1,000 loops each)
Temps d'exécution pour lopezpaul:
393 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1,000 loops each)
Temps d'exécution pour petersencasey:
444 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1,000 loops each)


In [6]:
cursor = conn.cursor()

cursor.execute("CREATE INDEX idx_username ON user_for_comparison (username);")
conn.commit()


In [10]:
# Récupérer 10 utilisateurs de la table
cursor.execute('SELECT username FROM "user_for_comparison" LIMIT 5')
usernames_to_query = [row[0] for row in cursor.fetchall()]

for username in usernames_to_query:
    # Créer un nouveau curseur à chaque itération
    cursor = conn.cursor()

    query = f"SELECT * FROM user_for_comparison WHERE username = '{username}'"

    print(f"Temps d'exécution pour {username}:")
    %timeit -r 1 -o cursor.execute(query) 

    results = cursor.fetchall()


Temps d'exécution pour leejessica:
492 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1,000 loops each)
Temps d'exécution pour peterdougherty:
541 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1,000 loops each)
Temps d'exécution pour victoria59:
870 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1,000 loops each)
Temps d'exécution pour lopezpaul:
632 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1,000 loops each)
Temps d'exécution pour petersencasey:
724 µs ± 0 ns per loop (mean ± std. dev. of 1 run, 1,000 loops each)


## Fermer le curseur et la connexion

In [11]:
# Fermer le curseur et la connexion
cursor.close()
conn.close()